In [14]:
from  selenium import webdriver 
from bs4 import BeautifulSoup as bs
import re 
import string
from time import  sleep
from selenium.common.exceptions import TimeoutException
import pickle
from collections import deque
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
with open("./vilas_state.pkl",'rb') as file:
        vilas,stack2,link2=pickle.load(file)
df_vilas=pd.DataFrame(vilas)

In [16]:
with open("./vlias-for-rent.pkl",'rb') as file:
        vilas_rent,stack2,link2=pickle.load(file)
df_vilas_rent=pd.DataFrame(vilas_rent)

In [17]:
with open("./apartmant.pkl",'rb') as file:
        apartmant,stack2,link2=pickle.load(file)
df_apartmant=pd.DataFrame(apartmant)

In [18]:
with open("./apertment-rent.pkl",'rb') as file:
        apertment_rent,stack2,link2=pickle.load(file)
df_apertment_rent=pd.DataFrame(apertment_rent)

In [19]:
with open("state21.pkl",'rb') as file:
        state21,stack2,link2=pickle.load(file)
df_state21=pd.DataFrame(state21)